In [1]:
from bs4 import BeautifulSoup
import requests
import hospital 

In [ ]:
class ResourceType(Enum):
    BED_WITHOUT_OXYGEN = "BED_WITHOUT_OXYGEN"
    BED_WITH_OXYGEN = "BED_WITH_OXYGEN"
    ICU_WITH_VENTILATOR = "ICU_WITH_VENTILATOR"
    ICU_WITHOUT_VENTILATOR = "ICU_WITHOUT_VENTILATOR"
    TOTAL_BEDS = "TOTAL_BEDS"


@dataclass
class Resource:
    resource_type: ResourceType
    resource_description: str
    resource_qty: int


@dataclass
class Hospital:
    name: str
    address: str
    district: str
    city: str
    state: str
    location: str
    last_updated: datetime
    resources: List[Resource]

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}
page = requests.get('https://pratirakshak.co.in/new-report.php', headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
COL_NAMES = ['hosp_name', 'hosp_contact', 'ic_contact', 'No_Oxygen_Total', 'No_Oxygen_Occupied', 'No_Oxygen_Available', 
                'Oxygen_Total', 'Oxygen_Occupied', 'Oxygen_Available',
                 'ICU_Total', 'ICU_Occupied', 'ICU_Available', 'last_updated'
            ]

In [4]:
hospital_data = []
tables = soup.find('table')
for row in tables.find_all('tr'):
    row_data = [d.text for d in row.find_all('td')]
    if not row_data:
        continue
    row_data = { c : r for r, c in zip(row_data, COL_NAMES) }
    resources = [
        hospital.Resource(hospital.ResourceType.BED_WITHOUT_OXYGEN, None, row_data['No_Oxygen_Available']),
        hospital.Resource(hospital.ResourceType.BED_WITH_OXYGEN, None, row_data['Oxygen_Available']),
        hospital.Resource(hospital.ResourceType.TOTAL_ICUS, None, row_data['ICU_Available'])
    ]
    hosp = hospital.Hospital(
        row_data['hosp_name'], None, 'Ranchi', 'Ranchi', 'Jharkhand', '', row_data['last_updated'], resources
    )
    hospital_data.append(hosp)
    
    
    

In [5]:
hospital_data

[Hospital(name='Medica', address=None, district='Ranchi', city='Ranchi', state='Jharkhand', location='', last_updated='2021-05-02 01:47:34', resources=[Resource(resource_type=<ResourceType.BED_WITHOUT_OXYGEN: 'BED_WITHOUT_OXYGEN'>, resource_description=None, resource_qty='0'), Resource(resource_type=<ResourceType.BED_WITH_OXYGEN: 'BED_WITH_OXYGEN'>, resource_description=None, resource_qty='0'), Resource(resource_type=<ResourceType.TOTAL_ICUS: 'TOTAL_ICUS'>, resource_description=None, resource_qty='0')]),
 Hospital(name='Alam Hospital Bariyatu', address=None, district='Ranchi', city='Ranchi', state='Jharkhand', location='', last_updated='2021-05-02 01:32:12', resources=[Resource(resource_type=<ResourceType.BED_WITHOUT_OXYGEN: 'BED_WITHOUT_OXYGEN'>, resource_description=None, resource_qty='0'), Resource(resource_type=<ResourceType.BED_WITH_OXYGEN: 'BED_WITH_OXYGEN'>, resource_description=None, resource_qty='0'), Resource(resource_type=<ResourceType.TOTAL_ICUS: 'TOTAL_ICUS'>, resource_des